In [2]:
#Start Part II.

In [3]:
# Import Dependencies
import pandas as pd 
import gmaps
import requests
import matplotlib.pyplot as plt 
from config import g_key

In [4]:
# Read CSV file and convert into a DataFrame
City_weather_df = pd.read_csv("Weather_Database/WeatherPY_database.csv")

# Configure gmaps to use api key
gmaps.configure(api_key=g_key)

In [5]:
# Input customer preferences
Pref_Min_Temp = float(input("What is the minimum temperature you prefer for this trip? "))
Pref_Max_Temp = float(input("What is the maximum temperature you prefer for this trip? "))
Rain_Pref = input("Do you prefer rain? (yes/no) ")
Snow_Pref = input("Do you prefer snow? (yes/no) ")

What is the minimum temperature you prefer for this trip? 70
What is the maximum temperature you prefer for this trip? 89
Do you prefer rain? (yes/no) no
Do you prefer snow? (yes/no) no


In [6]:
#filter dataframe based on customer inputs
preferred_City_df = City_weather_df.loc[(City_weather_df["Max Temp"] <= Pref_Max_Temp) & (City_weather_df["Max Temp"] >=Pref_Min_Temp)]

if Rain_Pref =="yes":
    preferred_City_df = preferred_City_df[preferred_City_df["Rain (inches)"] !=0]
else:
    preferred_City_df = preferred_City_df[preferred_City_df["Rain (inches)"] ==0]
if Snow_Pref == "yes":
    preferred_City_df = preferred_City_df[preferred_City_df["Snow (inches)"] !=0]
else:
    preferred_City_df = preferred_City_df[preferred_City_df["Snow (inches)"] ==0]

In [7]:
#create hotel name column
preferred_City_df["Hotel Name"] = ""

In [8]:
#hotel search parameters
params = {"radius": 5000, "type":"lodging", "key":g_key}
#iterate thru the dataframe
for index, row in preferred_City_df.iterrows():
    #get lat and lng
    lat = row["Lat"]
    lng = row["Lng"]
    #add lat and lng to location key for params dict
    params["location"] = f"{lat},{lng}"
    #use search term lodging and lat and lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #make request and get the json data from search
    hotels = requests.get(base_url, params=params).json()
    #grab first hotel from the results and store the name
    try:
        preferred_City_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found....skipping")

Hotel not found....skipping


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
# Create an InfoBox
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [ ]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in preferred_City_df.iterrows()]

In [ ]:
# Create a heatmap of temp for vacation cities
locations =  preferred_City_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

In [ ]:
#create output file (csv)
output_data_file = "weather_data/weatherPy_vacation.csv"
#export city_data into a csv
preferred_City_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
preferred_City_df